학습할 파라메터없이 입력을 받아 함수를 사용해 출력해주는 Lambda layer 에 대해 공부해보겠습니다.

In [1]:
from tensorflow.keras import layers, models
import tensorflow.keras.backend as K
import numpy as np

x_test = np.array([[1,2,3], [4,5,6]])
x = layers.Input(shape=(3,))

모델 summary 를 보면 Lambda 층에는 학습할 파라메터가 없는것을 알 수 있습니다.

In [2]:
def func(x):
    return x * 2

y = layers.Lambda(func)(x)
model = models.Model(x, y)
model.summary()
y_pred = model.predict(x_test)

print()
print('x_test'); print(x_test)
print()
print('y_pred'); print(y_pred)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
lambda (Lambda)              (None, 3)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

x_test
[[1 2 3]
 [4 5 6]]

y_pred
[[ 2.  4.  6.]
 [ 8. 10. 12.]]


간단한 한줄함수는 아래와 같이 lambda 함수를 넣어줄 수 있습니다.

In [3]:
y = layers.Lambda(lambda x: x**2)(x)
model = models.Model(x, y)
model.summary()
y_pred = model.predict(x_test)

print()
print('x_test'); print(x_test)
print()
print('y_pred'); print(y_pred)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 3)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

x_test
[[1 2 3]
 [4 5 6]]

y_pred
[[ 1.  4.  9.]
 [16. 25. 36.]]


아래와 같이 람다 레이어 안에서 K.sum 같은 keras.backend 함수를 사용할 수도 있습니다.

In [4]:
def func(x):
    # axis 0 is the batch dimension
    return K.sum(x, axis=1, keepdims=True)

y = layers.Lambda(func)(x)
model = models.Model(x, y)
model.summary()
y_pred = model.predict(x_test)

print()
print('x_test'); print(x_test)
print()
print('y_pred'); print(y_pred)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 1)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

x_test
[[1 2 3]
 [4 5 6]]

y_pred
[[ 6.]
 [15.]]


조건에 따라 다르게 계산하려면 Python 의 조건문 대신 backend.switch 함수를 사용합니다.

In [5]:
#y = layers.Lambda(lambda x: 4 if x>4 else x)(x) does not work
y = layers.Lambda(lambda x: K.switch(x>4, 4*K.ones_like(x), x))(x)
model = models.Model(x, y)
model.summary()
y_pred = model.predict(x_test)

print()
print('x_test'); print(x_test)
print()
print('y_pred'); print(y_pred)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 3)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

x_test
[[1 2 3]
 [4 5 6]]

y_pred
[[1. 2. 3.]
 [4. 4. 4.]]


아래와 같이 리스트 형식으로 여러개의 텐서를 출력할 수 있습니다.

In [6]:
yA, yB = layers.Lambda(lambda x: [x/2, x*2])(x)
model = models.Model(x, [yA, yB])
model.summary()
y_pred_A, y_pred_B = model.predict(x_test)

print()
print('x_test'); print(x_test)
print()
print('y_pred_A'); print(y_pred_A)
print()
print('y_pred_B'); print(y_pred_B)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
lambda_4 (Lambda)            [(None, 3), (None, 3)]    0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

x_test
[[1 2 3]
 [4 5 6]]

y_pred_A
[[0.5 1.  1.5]
 [2.  2.5 3. ]]

y_pred_B
[[ 2.  4.  6.]
 [ 8. 10. 12.]]


아래와 같이 입력 텐서를 여러 텐서로 나누어 출력할 수 있습니다.

In [7]:
yA, yB = layers.Lambda(lambda x: [x[:,0:2], x[:,2:]])(x)
model = models.Model(x, [yA, yB])
model.summary()
y_pred_A, y_pred_B = model.predict(x_test)

print()
print('x_test'); print(x_test)
print()
print('y_pred_A'); print(y_pred_A)
print()
print('y_pred_B'); print(y_pred_B)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
lambda_5 (Lambda)            [(None, 2), (None, 1)]    0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

x_test
[[1 2 3]
 [4 5 6]]

y_pred_A
[[1. 2.]
 [4. 5.]]

y_pred_B
[[3.]
 [6.]]


activation 함수들을 backend.exp 와 backend.switch 함수를 사용하여 Lambda layer 로 대체해보겠습니다

In [8]:
from tensorflow.keras import layers, models
import tensorflow.keras.backend as K
import numpy as np

x_test = np.array([[-1],[2]])
x = layers.Input((1,))

sigmoid 함수를 Lambda layer 로 대체해 보겠습니다.

In [9]:
y = layers.Activation('sigmoid')(x)
model = models.Model(x,y)
model.summary()
y_pred = model.predict(x_test)

print()
print('y_pred');print(y_pred)
print()

y = layers.Lambda(lambda x: 1/(1+K.exp(-x)))(x)
model = models.Model(x,y)
model.summary()
y_pred = model.predict(x_test)

print()
print('y_pred'); print(y_pred)

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

y_pred
[[0.26894143]
 [0.880797  ]]

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda_6 (Lambda)            (None, 1)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

y_pred
[[0.2689

tanh 함수를 Lambda layer 로 대체해 보겠습니다.

In [10]:
y = layers.Activation('tanh')(x)
model = models.Model(x,y)
model.summary()
y_pred = model.predict(x_test)

print()
print('y_pred'); print(y_pred)
print()

y = layers.Lambda(lambda x: (1-K.exp(-2*x))/(1+K.exp(-2*x)))(x)
model = models.Model(x,y)
model.summary()
y_pred = model.predict(x_test)

print()
print('y_pred'); print(y_pred)

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

y_pred
[[-0.7615942]
 [ 0.9640276]]

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda_7 (Lambda)            (None, 1)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

y_pred
[[-0.761

relu 함수를 Lambda layer 로 대체해 보겠습니다.

In [11]:
y = layers.Activation('relu')(x)
model = models.Model(x,y)
model.summary()
y_pred = model.predict(x_test)

print()
print('y_pred'); print(y_pred)
print()

y = layers.Lambda(lambda x: K.switch(x>0, x, K.zeros_like(x)))(x)
model = models.Model(x,y)
model.summary()
y_pred = model.predict(x_test)

print()
print('y_pred'); print(y_pred)

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

y_pred
[[0.]
 [2.]]

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda_8 (Lambda)            (None, None)              0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

y_pred
[[0.]
 [2.]]
